## **Supervised ML regression algorithm to predict next round team value (CT & T)**
## **Algorithm**

In [1]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)

### Data

In [3]:
ct_df = pd.read_csv('../data/processed/4_base_predict_next_rnd_ct_type.csv')
t_df = pd.read_csv('../data/processed/4_base_predict_next_rnd_t_type.csv')

In [4]:
display(ct_df.head())
display(t_df.head())

,file,round,wp_ct_val,nade_ct_val,ct_alive,t_alive,ct_winner,bomb_planted,ct_cons_wins,t_cons_wins,ct_val_pred,round_type,nxt_rnd_type
0,0,1,1000.0,550,5,5,0.5,0.5,0,0,4078.134589,PISTOL_ROUND,MEDIUM
1,0,2,10100.0,1100,4,0,1.0,0.0,1,0,17819.702711,MEDIUM,MEDIUM
2,0,3,4125.0,900,0,1,0.0,0.0,0,1,7038.468589,MEDIUM,ECO
3,0,4,1000.0,0,0,3,0.0,1.0,0,2,1452.468928,ECO,FULL
4,0,5,15500.0,1400,0,4,0.0,1.0,0,3,22676.205763,FULL,FULL


,file,round,wp_t_val,nade_t_val,ct_alive,t_alive,ct_winner,bomb_planted,ct_cons_wins,t_cons_wins,t_val_pred,round_type,nxt_rnd_type
0,0,1,1166.666667,1200,5,5,0.5,0.5,0,0,3943.272665,PISTOL_ROUND,ECO
1,0,2,3687.500000,50,4,0,1.0,0.0,1,0,6290.616771,ECO,FULL
2,0,3,11700.000000,2450,0,1,0.0,0.0,0,1,19600.790638,FULL,MEDIUM
3,0,4,11700.000000,1600,0,3,0.0,1.0,0,2,22568.098741,MEDIUM,FULL
4,0,5,12750.000000,1700,0,4,0.0,1.0,0,3,24459.855175,FULL,MEDIUM


### Encoding

We use a manual encoding to be sure of being the same values for round_type and nxt_rnd_type.

If we use LabelEncoder() it could be possible to have different values for the same label.

In [5]:
ct_df['round_type'].unique()

array(['PISTOL_ROUND', 'MEDIUM', 'ECO', 'FULL'], dtype=object)

In [6]:
round_type_dic = {'PISTOL_ROUND':0, 'ECO':1, 'MEDIUM':2, 'FULL':3, 'LAST':4}

In [7]:
ct_df['round_type'] = ct_df['round_type'].apply(lambda x: round_type_dic[x])
ct_df['nxt_rnd_type'] = ct_df['nxt_rnd_type'].apply(lambda x: round_type_dic[x])

t_df['round_type'] = t_df['round_type'].apply(lambda x: round_type_dic[x])
t_df['nxt_rnd_type'] = t_df['nxt_rnd_type'].apply(lambda x: round_type_dic[x])

In [8]:
display(ct_df.head())
display(t_df.head())

,file,round,wp_ct_val,nade_ct_val,ct_alive,t_alive,ct_winner,bomb_planted,ct_cons_wins,t_cons_wins,ct_val_pred,round_type,nxt_rnd_type
0,0,1,1000.0,550,5,5,0.5,0.5,0,0,4078.134589,0,2
1,0,2,10100.0,1100,4,0,1.0,0.0,1,0,17819.702711,2,2
2,0,3,4125.0,900,0,1,0.0,0.0,0,1,7038.468589,2,1
3,0,4,1000.0,0,0,3,0.0,1.0,0,2,1452.468928,1,3
4,0,5,15500.0,1400,0,4,0.0,1.0,0,3,22676.205763,3,3


,file,round,wp_t_val,nade_t_val,ct_alive,t_alive,ct_winner,bomb_planted,ct_cons_wins,t_cons_wins,t_val_pred,round_type,nxt_rnd_type
0,0,1,1166.666667,1200,5,5,0.5,0.5,0,0,3943.272665,0,1
1,0,2,3687.500000,50,4,0,1.0,0.0,1,0,6290.616771,1,3
2,0,3,11700.000000,2450,0,1,0.0,0.0,0,1,19600.790638,3,2
3,0,4,11700.000000,1600,0,3,0.0,1.0,0,2,22568.098741,2,3
4,0,5,12750.000000,1700,0,4,0.0,1.0,0,3,24459.855175,3,2


## Define features and target

In [9]:
CT_FEATS = ['file', 'round', 'wp_ct_val', 'nade_ct_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 'ct_val_pred', 'round_type']
T_FEATS = ['file', 'round', 'wp_t_val', 'nade_t_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 't_val_pred', 'round_type']
TARGET = 'nxt_rnd_type'

In [10]:
#Preprocessor

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

preprocessor_ct = ColumnTransformer(transformers=[('num', numeric_transformer, CT_FEATS)])
preprocessor_t = ColumnTransformer(transformers=[('num', numeric_transformer, T_FEATS)])

### **SPLITS**

In [11]:
ct_train, ct_test = train_test_split(ct_df)
t_train, t_test = train_test_split(t_df)

In [12]:
print(ct_train.shape, ct_test.shape)
print(t_train.shape, t_test.shape)

(233109, 13) (77704, 13)
(233109, 13) (77704, 13)


### **MODEL**

In [13]:
# regressor = LGBMRegressor(boosting_type='gbdt', 
#                        bagging_freq=1, 
#                        bagging_fraction = 0.9, 
#                        n_estimators=100)

regressor = RandomForestClassifier()

In [14]:
ct_model = Pipeline(steps=[('ct_preprocessor', preprocessor_ct),
                           ('regressor', regressor)])

# t_model = Pipeline(steps=[('t_preprocessor', preprocessor_t),
#                            ('regressor', regressor)])

In [15]:
ct_model.fit(ct_train[CT_FEATS], ct_train[TARGET]);

# t_model.fit(t_train[T_FEATS], t_train[TARGET]);

In [16]:
y_ct_test = ct_model.predict_proba(ct_test[CT_FEATS])[:,1]
y_ct_train = ct_model.predict_proba(ct_train[CT_FEATS])[:,1]

# y_ct_test = ct_model.predict(ct_test[CT_FEATS])
# y_ct_train = ct_model.predict(ct_train[CT_FEATS])

# y_t_test = t_model.predict(t_test[T_FEATS])
# y_t_train = t_model.predict(t_train[T_FEATS])

### **CHECK PERFORMANCE**

In [20]:
# 

print(f"CT test error: {roc_auc_score(ct_test[TARGET].to_numpy(), y_ct_test, average = 'macro', multi_class='ovr')}")
print(f"CT train error: {roc_auc_score(ct_train[TARGET].to_numpy(), y_ct_train, average = 'macro', multi_class='ovr')}")
print()
# print(f"T test error: {roc_auc_score(y_pred=y_t_test, y_true=t_test[T_TARGET])}")
# print(f"T train error: {roc_auc_score(y_pred=y_t_train, y_true=t_train[T_TARGET])}")

AxisError: axis 1 is out of bounds for array of dimension 1